In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Import the dataset

In [2]:
train_file = r'loan_data_train.csv'
test_file = r'loan_data_test.csv'

ld_train = pd.read_csv(train_file)
ld_test = pd.read_csv(test_file)

In [3]:
ld_train.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [4]:
ld_test.shape

(300, 14)

In [5]:
ld_test['Interest.Rate'] = np.nan

ld_train['data'] = 'train'
ld_test['data'] = 'test'

ld_test = ld_test[ld_train.columns]
ld_all = pd.concat([ld_train, ld_test], axis=0)

In [6]:
ld_all.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years,train
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years,train
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year,train
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years,train
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years,train


In [7]:
ld_all.dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
data                               object
dtype: object

In [8]:
#ID, Amount.Funded.By.Investors : drop columns
#Interest.Rate, Debt.To.Income.Ratio : remove the % and convert to numeric
#Amount.Requested, Open.CREDIT.Lines, Revolving.CREDIT.Balance : convert to numeric
#FICO.Range : replace it by an average of the range
#Employment.Length : convert to numeric
#Loan.Length, Loan.Purpose, State , Home.Ownership : convert to dummies

Dropping Redundant columns

In [9]:
ld_all.drop(['ID', 'Amount.Funded.By.Investors'],axis=1,inplace = True)

In [10]:
for col in ['Interest.Rate', 'Debt.To.Income.Ratio']:
    ld_all[col] = ld_all[col].str.replace("%","")

In [11]:
ld_all[['Interest.Rate', 'Debt.To.Income.Ratio']].head()

,Interest.Rate,Debt.To.Income.Ratio
0,18.49,27.56
1,17.27,13.39
2,14.33,3.50
3,16.29,19.62
4,12.23,23.79


In [12]:
for col in ['Amount.Requested', 'Open.CREDIT.Lines', 'Revolving.CREDIT.Balance',
            'Interest.Rate', 'Debt.To.Income.Ratio']:
    ld_all[col] = pd.to_numeric(ld_all[col], errors='coerce')

In [13]:
k = ld_all['FICO.Range'].str.split('-', expand=True).astype(float)
ld_all['fico'] = (k[0]+k[1])*0.5
del ld_all['FICO.Range']

In [14]:
ld_all.head()

,Amount.Requested,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data,fico
0,25000.0,18.49,60 months,debt_consolidation,27.56,VA,MORTGAGE,8606.56,11.0,15210.0,3.0,5 years,train,722.0
1,19750.0,17.27,60 months,debt_consolidation,13.39,NY,MORTGAGE,6737.50,14.0,19070.0,3.0,4 years,train,712.0
2,2100.0,14.33,36 months,major_purchase,3.50,LA,OWN,1000.00,13.0,893.0,1.0,< 1 year,train,692.0
3,28000.0,16.29,36 months,credit_card,19.62,NV,MORTGAGE,7083.33,12.0,38194.0,1.0,10+ years,train,712.0
4,24250.0,12.23,60 months,credit_card,23.79,OH,MORTGAGE,5833.33,6.0,31061.0,2.0,10+ years,train,732.0


In [15]:
ld_all['Employment.Length'].value_counts()

10+ years    653
< 1 year     249
2 years      243
3 years      235
5 years      202
4 years      191
1 year       177
6 years      163
7 years      127
8 years      108
9 years       72
.              2
Name: Employment.Length, dtype: int64

In [16]:
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace('years',"")
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace('year',"")

ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[:2]=='10',10,ld_all['Employment.Length'])
ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[:1]=='<',0,ld_all['Employment.Length'])
ld_all['Employment.Length'] = pd.to_numeric(ld_all['Employment.Length'], errors = 'coerce')

Get list of columns for creating dummy variables

In [17]:
cat_cols = ld_all.select_dtypes(['object']).columns
cat_cols

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership', 'data'], dtype='object')

In [18]:
for col in cat_cols[:-1]:
    dummy = pd.get_dummies(ld_all[col], drop_first=True, prefix = col)
    ld_all=pd.concat([ld_all,dummy],axis=1)
    del ld_all[col]
    print(col)
del dummy

Loan.Length
Loan.Purpose
State
Home.Ownership


In [19]:
cat_cols

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership', 'data'], dtype='object')

In [20]:
ld_all.shape

(2500, 75)

In [21]:
ld_all.isnull().sum()

Amount.Requested          5
Interest.Rate           300
Debt.To.Income.Ratio      1
Monthly.Income            3
Open.CREDIT.Lines         9
                       ... 
State_WY                  0
Home.Ownership_NONE       0
Home.Ownership_OTHER      0
Home.Ownership_OWN        0
Home.Ownership_RENT       0
Length: 75, dtype: int64

In [22]:
for col in ld_all.columns:
    if (col not in ['Interest.Rate','data']) & (ld_all[col].isnull().sum()>0):
        ld_all.loc[ld_all[col].isnull(),col]=ld_all.loc[ld_all['data']=='train',col].mean()

In [23]:
for col in ld_all.columns:
    print(col+" "+str(ld_all[col].isnull().sum()))

Amount.Requested 0
Interest.Rate 300
Debt.To.Income.Ratio 0
Monthly.Income 0
Open.CREDIT.Lines 0
Revolving.CREDIT.Balance 0
Inquiries.in.the.Last.6.Months 0
Employment.Length 0
data 0
fico 0
Loan.Length_36 months 0
Loan.Length_60 months 0
Loan.Purpose_credit_card 0
Loan.Purpose_debt_consolidation 0
Loan.Purpose_educational 0
Loan.Purpose_home_improvement 0
Loan.Purpose_house 0
Loan.Purpose_major_purchase 0
Loan.Purpose_medical 0
Loan.Purpose_moving 0
Loan.Purpose_other 0
Loan.Purpose_renewable_energy 0
Loan.Purpose_small_business 0
Loan.Purpose_vacation 0
Loan.Purpose_wedding 0
State_AK 0
State_AL 0
State_AR 0
State_AZ 0
State_CA 0
State_CO 0
State_CT 0
State_DC 0
State_DE 0
State_FL 0
State_GA 0
State_HI 0
State_IA 0
State_IL 0
State_IN 0
State_KS 0
State_KY 0
State_LA 0
State_MA 0
State_MD 0
State_MI 0
State_MN 0
State_MO 0
State_MS 0
State_MT 0
State_NC 0
State_NH 0
State_NJ 0
State_NM 0
State_NV 0
State_NY 0
State_OH 0
State_OK 0
State_OR 0
State_PA 0
State_RI 0
State_SC 0
State_SD

In [24]:
ld_train = ld_all[ld_all['data']=='train']
del ld_train['data']
ld_test = ld_all[ld_all['data']=='test']
ld_test.drop(['Interest.Rate','data'],axis=1,inplace=True)

/home/glenshel/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
del ld_all

In [26]:
ld_train.shape, ld_test.shape

((2200, 74), (300, 73))

In [27]:
ld_train1, ld_train2 = train_test_split(ld_train, test_size = 0.2, random_state = 2)

In [28]:
x_train1 = ld_train1.drop('Interest.Rate', axis=1)
y_train1 = ld_train1['Interest.Rate']

In [29]:
lm = LinearRegression()

In [30]:
lm.fit(x_train1, y_train1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
lm.intercept_

73.14080030298322

In [32]:
list(zip(x_train1.columns, lm.coef_))

[('Amount.Requested', 0.00015484548002310893),
 ('Debt.To.Income.Ratio', -0.002298334101124195),
 ('Monthly.Income', -2.4033069223428576e-05),
 ('Open.CREDIT.Lines', -0.04083407389413675),
 ('Revolving.CREDIT.Balance', -4.462373135893927e-06),
 ('Inquiries.in.the.Last.6.Months', 0.3432576476536704),
 ('Employment.Length', 0.03242527420900587),
 ('fico', -0.0865836799225371),
 ('Loan.Length_36 months', 1.164108764186322),
 ('Loan.Length_60 months', 4.326606348529703),
 ('Loan.Purpose_credit_card', -0.5964208918178797),
 ('Loan.Purpose_debt_consolidation', -0.42642141395138067),
 ('Loan.Purpose_educational', 0.15648274455613184),
 ('Loan.Purpose_home_improvement', -0.3225421936138716),
 ('Loan.Purpose_house', 0.18469552938412956),
 ('Loan.Purpose_major_purchase', -0.09544292596209482),
 ('Loan.Purpose_medical', -0.37330179997081836),
 ('Loan.Purpose_moving', 1.2660338147822179),
 ('Loan.Purpose_other', 0.4420603478110547),
 ('Loan.Purpose_renewable_energy', -0.11783058981536133),
 ('Loan

In [33]:
x_train2 = ld_train2.drop('Interest.Rate', axis=1)

In [34]:
predicted_ir = lm.predict(x_train2)

In [35]:
from sklearn.metrics import mean_absolute_error

In [36]:
mean_absolute_error(ld_train2['Interest.Rate'], predicted_ir)

1.6223891455793797

Prediction for the entire training dataset to make prediction on test

In [37]:
x_train = ld_train.drop('Interest.Rate', axis=1)
y_train = ld_train['Interest.Rate']

In [38]:
lm.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [39]:
test_pred = lm.predict(ld_test)

Write the output to a csv file

In [40]:
pd.DataFrame(test_pred).to_csv('output.csv', index=False)

# Ridge Regression using GridSearchCV

In [41]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [42]:
lambdas = np.linspace(1,100,100)
params = {'alpha' : lambdas}

In [43]:
model = Ridge()

In [44]:
grid_search = GridSearchCV(model, 
                           param_grid = params, 
                           cv = 10,
                           scoring='neg_mean_absolute_error')

In [45]:
grid_search.fit(x_train1, y_train1)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,...
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.]

In [46]:
grid_search.best_params_

{'alpha': 57.0}

In [47]:
grid_search.best_estimator_

Ridge(alpha=57.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [48]:
model = Ridge(alpha=57.0)
model.fit(x_train1, y_train1)

Ridge(alpha=57.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [49]:
predicted_ir2 = model.predict(x_train2)

In [51]:
mean_absolute_error(ld_train2['Interest.Rate'], predicted_ir2)

1.6183228135120598